<a href="https://colab.research.google.com/github/Idan-Alter/OU-20936-Data-Science-Workshop/blob/main/20936_X_1_NLP_Tasks_with_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Workshop (20936)
## The Open University of Israel
### Department of Mathematics and Computer Science
####Idan Alter

### Unit X - ANN Applications with Hugging Face
#### Chapter 1 - NLP Tasks

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 48.3 MB/s 
     |████████████████████████████████| 120 kB 47.2 MB/s 


In [ ]:
import nltk
nltk.download("twitter_samples")
from nltk.corpus import twitter_samples

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


In [ ]:
pos_tweets = twitter_samples.strings('positive_tweets.json')
neg_tweets = twitter_samples.strings('negative_tweets.json')
print(pos_tweets[0],neg_tweets[0],sep="\n")

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
hopeless for tmr :(


In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
classifier("I'm very happy")

[{'label': 'POSITIVE', 'score': 0.9998770952224731}]

In [ ]:
classifier("I'm sometimes sad")

[{'label': 'NEGATIVE', 'score': 0.9977946281433105}]

In [ ]:
classifier("I'm sometimes sad, sometimes happy")

[{'label': 'POSITIVE', 'score': 0.998806357383728}]

In [ ]:
print(*pos_tweets[:5],sep="\n\n")
classifier(pos_tweets[:5])

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!

@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!

@97sides CONGRATS :)

yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days


[{'label': 'POSITIVE', 'score': 0.9260643720626831},
 {'label': 'POSITIVE', 'score': 0.9991611242294312},
 {'label': 'POSITIVE', 'score': 0.9996201992034912},
 {'label': 'POSITIVE', 'score': 0.9924265146255493},
 {'label': 'NEGATIVE', 'score': 0.9725946187973022}]

In [ ]:
def calc_acc(reslut,target_label):
  hits = 0
  for result_dict in result:
    if result_dict["label"] == target_label:
      hits += 1
  return hits/len(result)

In [ ]:
num_samples  = 1000
result       = classifier(pos_tweets[:num_samples])
pos_accuracy = calc_acc(result,target_label='POSITIVE')
print(pos_accuracy)

0.522


In [ ]:
result       = classifier(neg_tweets[:num_samples])
neg_accuracy = calc_acc(result,target_label='NEGATIVE')
print(neg_accuracy)

0.798


In [ ]:
!pip install emoji==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 3.2 MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49734 sha256=d084a4d4c808cf53a6fa25e070f4d5184d6ded876ee51613714c612a05cdd81b
  Stored in directory: /root/.cache/pip/wheels/4e/bf/6b/2e22b3708d14bf6384f862db539b044d6931bd6b14ad3c9adc
Successfully built emoji


In [ ]:
classifier   = pipeline("sentiment-analysis",
                model = "cardiffnlp/twitter-roberta-base-sentiment")
classifier([pos_tweets[0],neg_tweets[0]])

[{'label': 'LABEL_2', 'score': 0.9538478255271912},
 {'label': 'LABEL_0', 'score': 0.7271883487701416}]

In [ ]:
result       = classifier(pos_tweets[:num_samples])
pos_accuracy = calc_acc(result,target_label='LABEL_2')
print(pos_accuracy)

0.9


In [ ]:
result       = classifier(neg_tweets[:num_samples])
neg_accuracy = calc_acc(result,target_label='LABEL_0')
print(neg_accuracy)

0.786


### Feature Extraction

In [ ]:
feature_extractor = pipeline("feature-extraction")

No model was supplied, defaulted to distilbert-base-cased and revision 935ac13 (https://huggingface.co/distilbert-base-cased).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentence    = "HelloWorld Hello World"
model_input = feature_extractor.preprocess(sentence)
print(model_input)

{'input_ids': tensor([[ 101, 8667, 2924, 1766, 5253, 8667, 1291,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
decoded = feature_extractor.tokenizer.convert_ids_to_tokens(
          model_input["input_ids"][0])
print(decoded,sentence,sep="\n")

['[CLS]', 'Hello', '##W', '##or', '##ld', 'Hello', 'World', '[SEP]']
HelloWorld Hello World


In [ ]:
import pandas as pd
features = feature_extractor(sentence)
token_features = pd.DataFrame(features[0],index=decoded)
token_features.shape

(8, 768)

In [ ]:
sentence_features = token_features.mean(axis=0)
sentence_features.shape

(768,)